In [1]:
# Importing Libraries Required to import API's and Graph/Interface with Data:
import os
import numpy as np
import pandas as pd
import json as js
import requests
import http.client
from dotenv import load_dotenv
from pathlib import Path
from tkinter import *
from tkinter import ttk

The below step was included in order to load my .env file with the API key however, for ease of grading I have simply incorporated the API key within the code below so this code is no longer relevant.

In [2]:
# Loading .env environment variable:
#load_dotenv()

#### Extracting zip data and property data by Top 5 investment cities to be used in drop down list

In [3]:
# Defining csv paths to read Top Zip data and prop_type data csv which includes zip code and property type data:
zip_csv = Path("../../Data/zip_data.csv")

# imorting zip.csv to incorporate data into API, data selections:
zip_df = pd.read_csv(zip_csv)
zip_df.head()

,City,State,Zip Codes
0,Tampa,FL,33602
1,Tampa,FL,33603
2,Tampa,FL,33604
3,Tampa,FL,33605
4,Tampa,FL,33601


In [4]:
# setting zip_string, list variable and creating zip_list which includes zip info from df above:
zip_list = zip_df['Zip Codes'].tolist()

In [5]:
# building out dropdown of zip codes for user to make a selection and store in variable:
# creating Tkinter window:
master = Tk()
master.title("Select a Zip Code")
master.geometry("300x200")

# defining function that will store zip selection once button is clicked:
def selected():
    zips = myselect.get()
    print(zips)
    
    global zip_var
    zip_var = zips

# Creating combobox that stores zip data:
myselect = ttk.Combobox(master, values=zip_list)
myselect.current(0)
myselect.pack()

# Creating button that will print zip data and store in global variable:
mybutton = Button(master, text="Select", command=selected)
mybutton.pack()

master.mainloop()

33602
33605
33605
33614
33614
33684


The Cell below was required as I stored the API key within a .env file.
However, for ease of use for grading I have just added the API key to the code below. That being said, the cell below is not required in order to pull the API data.

In [10]:
# Setting API key for RAPID API Property Snapshot API:
rapid_api_key = os.getenv('RAPID_API_KEY')

# Verifying that key was passed through accurately:
print(f"Rapid Key type: {type(rapid_api_key)}")

Rapid Key type: <class 'NoneType'>


In [7]:
# Connecting Rapid API to my python code which houses sales data:
conn = http.client.HTTPSConnection("realtor.p.rapidapi.com")

# Creating headers as per the Rapid API documentation:
headers = {
    'x-rapidapi-key': 'faef5b5514msh2bba6e6b7604d8bp1a47a8jsn6b34037c7c72',
    'x-rapidapi-host': "realtor.p.rapidapi.com"
    }

# Initiating API request to pull property data from Rapid API:
conn.request("GET", f"/properties/v2/list-for-sale?limit=5&offset=0&sort=relevance&postal_code={zip_var}",
             headers=headers)

# assigning response to variable "res" and reading the data and assigning to variable "data"_
# Also formatting data response into JSON in order to read as JSON within an indented format:
res = conn.getresponse()
data = res.read().decode('utf-8')
json_obj = js.loads(data)
print(js.dumps(json_obj, indent=5))

{
     "meta": {
          "build": "3.23.123",
          "schema": "core.3",
          "tracking_params": {
               "channel": "for_sale",
               "siteSection": "for_sale",
               "city": "unknown",
               "county": "unknown",
               "neighborhood": "unknown",
               "searchCityState": "unknown",
               "state": "unknown",
               "zip": "33684",
               "srpPropertyStatus": "srp:for_sale",
               "listingActivity": "unknown",
               "propertyStatus": "for_sale",
               "propertyType": "any",
               "searchBathrooms": "any",
               "searchBedrooms": "any",
               "searchMaxPrice": "unknown",
               "searchMinPrice": "unknown",
               "searchRadius": "unknown",
               "searchHouseSqft": "any",
               "searchLotSqft": "any",
               "searchResults": "unknown",
               "sortResults": "relevance",
               "searchCoordinat

In [8]:
# Pulling data from top 5 properties to display useful sales data:
df_sale1 = pd.json_normalize(json_obj['properties'][0])
df_sale2 = pd.json_normalize(json_obj['properties'][1])
df_sale3 = pd.json_normalize(json_obj['properties'][2])
df_sale4 = pd.json_normalize(json_obj['properties'][3])
df_sale5 = pd.json_normalize(json_obj['properties'][4])

# combining sales data into single dataframe:
top_5_sale = pd.concat([df_sale1, df_sale2, df_sale3, df_sale4, df_sale5])

# Selecting columns that are relevant:
top_5_show = top_5_sale[['prop_type','prop_status','price','baths_full','baths','beds','thumbnail','lot_size.units',
                        'building_size.size','baths_half','mls.abbreviation']]
top_5_show

IndexError: list index out of range

In [ ]:
# Pulling address data from top 5 properties for sale:
df_add1 = pd.json_normalize(json_obj['properties'][0]['address'])
df_add2 = pd.json_normalize(json_obj['properties'][1]['address'])
df_add3 = pd.json_normalize(json_obj['properties'][2]['address'])
df_add4 = pd.json_normalize(json_obj['properties'][3]['address'])
df_add5 = pd.json_normalize(json_obj['properties'][4]['address'])

# Updating column names for readability:
top_5_add = pd.concat([df_add1, df_add2, df_add3, df_add4, df_add5])
top_5_add.drop(columns=['fips_code'], inplace=True)
top_5_add.columns = ['City', 'Address', 'Postal Code', 'State Code', 'State', 'County','Lattitude','Longitude',
                     'Neighborhood']
top_5_add